---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="496978f4" -->
# Table des matières
1. [Initialisation exemple](#Initialisation-exemple)
1. [Quatrième pipeline](#Quatrième-pipeline)
    1. [Définition de pipeline](#Définition-de-pipeline)
    1. [Définition de la recherche sur grille](#Définition-de-la-recherche-sur-grille)
    1. [Entraînement du pipeline avec chaque combinaison de paramètres](#Entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres)
    1. [Affichez le meilleur choix de paramètres](#Affichez-le-meilleur-choix-de-paramètres)
  
# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, decomposition, discriminant_analysis, manifold
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, QuantileTransformer, StandardScaler
from sklearn.svm import SVC

# Élimination des avertissements (warnings). On en observe régulièrement lorsqu'on utilise
# une méthode de recherche d'hyperparamètres. Plusieurs des entraînements associés
# ne convergent pas et un message d'avertissement est affiché pour chacun.
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

seed = 42

rng_seed = seed



<!-- #region id="bfee40f1" -->
Dans cet exemple, on reprend le même scénario que dans le module sur la conception d'un pipeline de préentrainement des données.
<!-- #endregion -->

<!-- #region id="70fa1d81" -->
# <a id=Initialisation-exemple>Initialisation exemple</a>
<!-- #endregion -->



In [ ]:
# Lecture du jeu de données et séparation de celles-ci en ensembles d'entraînement et de test

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30
n_components = 2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=rng_seed
)

def afficheMeilleurChoixParametres(grid_search):
    print("\nMeilleur choix de paramètres:")
    steps = dict(grid_search.best_estimator_.steps)

    for param_name in sorted(steps.keys()):
        print("\t%s: %r" % (param_name, steps[param_name]))

    print("\nScore optimal en entraînement: %0.1f %%" % (100 * grid_search.best_score_))
    print(
        "\nScore en test avec le pipeline optimal: %0.1f %%\n"
        % (100 * grid_search.score(X_test, y_test))
    )



<!-- #region id="db248117" -->
# <a id=Quatrième-pipeline>Quatrième pipeline</a>
<!-- #endregion -->

<!-- #region id="8d75c20b" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/pipeline-photo4.jpeg"  width="400" />
    <div>
    <font size="1.5">Image Source: http://www.brpetro.com/syllpx/</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="8dda5dba" -->
Dans cet exemple, nous utiliserons la normalisation standard comme méthode de transformation de données.
Nous allons optimiser la méthode de réduction de la dimensionnalité et les paramètres de la fonction SVM.

Les données en 64-D seront réduites en 2-D ou en 3-D (`n_components` $= [2, 3]$).

On va optimiser le pipeline en faisant varier les facteurs suivants:


- Réduction de la dimensionnalité:
    Méthodes:
    - PCA,
    - `LinearDiscriminantAnalysis`,
    - Isomap,
    - `n_components` $= [2, 3]$.
- Classificateur SVM:
    - `C` $= [1, 10, 100, 1000]$,
    - `gamma` $= [1, 0.1, 0.01, 0.001]$.



<!-- #endregion -->

<!-- #region id="2301ab97" -->
## <a id=Définition-de-pipeline>Définition de pipeline</a>
<!-- #endregion -->

<!-- #region id="1ea1b0ad" -->
Puisqu'il y a trois méthodes de réduction de la dimensionnalité, la grille des paramètres doit contenir trois dictionnaires de paramètres. Il y a maintenant trois mini pipelines.

Quelle serait la définition du pipeline?
<!-- #endregion -->



In [ ]:
pipeline = Pipeline(
    [
        ("transformation", StandardScaler()),
        ("reduce_dim", "passthrough"),
        ("classify", SVC(kernel="rbf", max_iter=1000)),
    ]
)

param_grid = [
    {
        "reduce_dim": [decomposition.PCA()],
        "reduce_dim__n_components": [2, 3],
        "classify__C": [1, 10, 100, 1000],
        "classify__gamma": [1, 0.1, 0.01, 0.001],
    },
    {
        "reduce_dim": [discriminant_analysis.LinearDiscriminantAnalysis()],
        "reduce_dim__n_components": [2, 3],
        "classify__C": [1, 10, 100, 1000],
        "classify__gamma": [1, 0.1, 0.01, 0.001],
    },
    {
        "reduce_dim": [manifold.Isomap()],
        "reduce_dim__n_components": [2, 3],
        "classify__C": [1, 10, 100, 1000],
        "classify__gamma": [1, 0.1, 0.01, 0.001],
    },
]



<!-- #region id="200fc94b" -->
## <a id=Définition-de-la-recherche-sur-grille>Définition de la recherche sur grille</a>
<!-- #endregion -->

<!-- #region id="d602439a" -->
Utilisez la fonction `GridSearchCV` pour initialiser l'optimisation des paramètres sur la grille.
<!-- #endregion -->



In [ ]:
grid_search = GridSearchCV(
    pipeline, param_grid=param_grid, n_jobs=2, verbose=1, refit=True
)



<!-- #region id="710b1126" -->
## <a id=Entraînement-du-pipeline-avec-chaque-combinaison-de-paramètres>Entraînement du pipeline avec chaque combinaison de paramètres</a>
<!-- #endregion -->

<!-- #region id="85e4f8fd" -->
Combien y a-t-il de modèles à entraîner?

**Réponse: L'affichage qui suit indique que 480 entraînements ont été effectués. Il y avait 3 modèles à tester, $2\times4\times4=32$
combinaisons d'hyperparamètres pour chacun, et 5 repliements pour chacun lors de la validation croisée,
soit un total de $3\times32\times5=480$ entraînements.**
<!-- #endregion -->

<!-- #region id="0b665dd4" -->
Effectuer l'entraînement du pipeline.
<!-- #endregion -->



In [ ]:
grid_search.fit(X_train, y_train)



<!-- #region id="c9f66a10" -->
## <a id=Affichez-le-meilleur-choix-de-paramètres>Affichez le meilleur choix de paramètres</a>
<!-- #endregion -->



In [ ]:
afficheMeilleurChoixParametres(grid_search)



<!-- #region id="3c43ee47" -->
Quelles conclusions tirez-vous (rappel la meilleure performance était de $98,9~\%$)?
<!-- #endregion -->

<!-- #region id="044db902" -->
**Réponse: Le quatrième pipeline est légèrement moins performant ($98,2~\%$) que le précédent ($98,9~\%$). Toutefois, on utilise ici un espace en 3-D plutôt qu'en 16-D. Donc, les performances sont relativement moins performantes, mais la complexité du modèle est plus faible. On remarque aussi que le score en test ($98,2~\%$) est supérieur à celui en entraînement ($97,5~\%$). Cela peut signifier deux choses, soit qu'il y a des fluctuations statistiques ou que notre modèle a fait du sous-apprentissage. C'est-à-dire, qu'il n'a pas maximisé son apprentissage sur le jeu de données.**
<!-- #endregion -->
